# Etapa 3 — PARTE 2 (Grupo 2)

Estimativa e análise da taxa de juros real.

- Janela inicial: 2019–2020
- Janela estendida: 2011–2020

Fonte de inflação: IPCA (aproximação). Fonte de taxa nominal: série compatível com as orientações (ex.: Selic/DI/curva a termo).


In [ ]:
# Imports e configuração
import os
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 160)
sns.set(style="whitegrid")

# Pastas de dados
DATA_DIR = os.path.join('data')
RAW_DIR = os.path.join(DATA_DIR, 'raw')
PROC_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

# Parâmetros do Grupo 2
WINDOWS = {
    '2019-2020': ('2019-01-01', '2020-12-31'),
    '2011-2020': ('2011-01-01', '2020-12-31'),
}

# Colunas esperadas
# ipca.csv: date(YYYY-MM), ipca_mom
# taxa_nominal.csv: date(YYYY-MM or YYYY-MM-DD), rate_nominal_aa (taxa nominal ao ano, em %)



In [ ]:
# Utilitários
from typing import Tuple

def to_month_start(date_series: pd.Series) -> pd.Series:
    s = pd.to_datetime(date_series)
    return s.values.astype('datetime64[M]').astype('datetime64[ns]')


def fisher_approx(nominal_aa: pd.Series, inflacao_aa: pd.Series) -> pd.Series:
    """Taxa real aproximada: r ≈ i - π (taxas anuais em fração).
    Entradas em % a.a.; saída em % a.a.
    """
    i = nominal_aa / 100.0
    pi = inflacao_aa / 100.0
    r = (i - pi) * 100.0
    return r


def fisher_exact(nominal_aa: pd.Series, inflacao_aa: pd.Series) -> pd.Series:
    """Taxa real exata: (1+r) = (1+i)/(1+π) ⇒ r = ((1+i)/(1+π) - 1)
    Entradas em % a.a.; saída em % a.a.
    """
    i = nominal_aa / 100.0
    pi = inflacao_aa / 100.0
    r = ((1.0 + i) / (1.0 + pi) - 1.0) * 100.0
    return r


def monthly_ipca_to_annualized(ipca_mom_pct: pd.Series) -> pd.Series:
    """Converte inflação mensal (%) para anualizada (%) via composição: (1+π_m)^12-1"""
    m = ipca_mom_pct / 100.0
    annual = ((1.0 + m) ** 12 - 1.0) * 100.0
    return annual



In [ ]:
# Carregar dados (ajuste os caminhos conforme sua estrutura)
# Esperado: data/raw/ipca.csv e data/raw/taxa_nominal.csv
ipca_path = os.path.join(RAW_DIR, 'ipca.csv')
nom_path = os.path.join(RAW_DIR, 'taxa_nominal.csv')

ipca = pd.read_csv(ipca_path)
nom = pd.read_csv(nom_path)

# Normalizar datas e colunas
if 'date' not in ipca.columns:
    raise ValueError("ipca.csv deve conter coluna 'date' (YYYY-MM) e 'ipca_mom' (% ao mês)")
if 'ipca_mom' not in ipca.columns:
    raise ValueError("ipca.csv deve conter coluna 'ipca_mom' (% ao mês)")

if 'date' not in nom.columns:
    raise ValueError("taxa_nominal.csv deve conter coluna 'date' e 'rate_nominal_aa' (% a.a.)")
if 'rate_nominal_aa' not in nom.columns:
    raise ValueError("taxa_nominal.csv deve conter coluna 'rate_nominal_aa' (% a.a.)")

ipca['date'] = to_month_start(ipca['date'])
nom['date'] = to_month_start(nom['date'])

# Anualizar IPCA
ipca['ipca_aa'] = monthly_ipca_to_annualized(ipca['ipca_mom'])

# Merge mensal (inner para meses em comum)
df = pd.merge(nom[['date', 'rate_nominal_aa']], ipca[['date', 'ipca_aa']], on='date', how='inner')
df = df.sort_values('date').reset_index(drop=True)

df.head()


In [ ]:
# Cálculo das taxas reais (aproximada e exata)
df['real_approx_aa'] = fisher_approx(df['rate_nominal_aa'], df['ipca_aa'])
df['real_exact_aa'] = fisher_exact(df['rate_nominal_aa'], df['ipca_aa'])

df[['date','rate_nominal_aa','ipca_aa','real_approx_aa','real_exact_aa']].tail()


In [ ]:
# Filtrar janelas e sumarizar
summaries = {}
for label, (start, end) in WINDOWS.items():
    mask = (df['date'] >= pd.to_datetime(start)) & (df['date'] <= pd.to_datetime(end))
    dsub = df.loc[mask].copy()
    summaries[label] = {
        'n_obs': len(dsub),
        'nominal_mean_aa': dsub['rate_nominal_aa'].mean(),
        'ipca_mean_aa': dsub['ipca_aa'].mean(),
        'real_approx_mean_aa': dsub['real_approx_aa'].mean(),
        'real_exact_mean_aa': dsub['real_exact_aa'].mean(),
    }

pd.DataFrame(summaries).T


In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axes[0].plot(df['date'], df['rate_nominal_aa'], label='Nominal (% a.a.)')
axes[0].plot(df['date'], df['ipca_aa'], label='IPCA anualizado (% a.a.)')
axes[0].set_title('Taxa nominal vs IPCA anualizado')
axes[0].legend()

axes[1].plot(df['date'], df['real_approx_aa'], label='Real aprox (% a.a.)')
axes[1].plot(df['date'], df['real_exact_aa'], label='Real exata (% a.a.)')
axes[1].set_title('Taxa real (aprox e exata)')
axes[1].legend()

plt.tight_layout()
plt.show()


## Notas
- Ajuste as fontes de dados conforme sua coleta (ex.: BCB/SGS, ANBIMA).
- Garanta frequência compatível (mensal) para o merge e coerência de prazos.
- Documente no `README.md` a origem e a data de extração das séries.
- Para reprodutibilidade, salve os CSVs em `data/raw/` com as colunas esperadas.
